

### William Fallas. williamfallas@gmail.com

#### Spech and translation with  Azure cognitive services

The following project shows how to extract audio from video and translate  the same from english to spanish


Main Steps

     -extract audio was format
    - Change stereo to mono
    - upload wav file to google bucket
    - Use the SpeechClient API, long_running_recognize method to get the english transcript
    - Use translate_v2 API to translate the transcript from english to spanish
    - Transform the spanish transcript to voice with  the texttospeech API 
    - Save the result in a mp3 file and play

In [1]:
!pip install azure-cognitiveservices-speech

In [8]:
import azure.cognitiveservices.speech as speechsdk

import moviepy
from pydub import AudioSegment
import io
import os
import wave
import time

In [4]:
def stereo_to_mono(audio_file_name):
    sound = AudioSegment.from_wav(audio_file_name)
    sound = sound.set_channels(1)
    sound.export(audio_file_name, format="wav")


def frame_rate_channel(audio_file_name):
    with wave.open(audio_file_name, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        channels = wave_file.getnchannels()
        return frame_rate,channels
    
    

In [35]:
pip install pytube

## Get Churchill speech from youtube

In [39]:
from pytube import YouTube
YouTube('https://www.youtube.com/watch?v=MkTw3_PmKtc').streams.first().download()

'C:\\Users\\wfc\\Documents\\IA\\CENFOTEC\\curso2 Aprendizaje Profundo\\Semana6-Cloud\\Semana6-Cloud\\Winston S Churchill We Shall Fight on the Beaches.mp4'

## Get Audio speech and save in .wav format

In [40]:
import moviepy.editor
video = moviepy.editor.VideoFileClip("Churchil.mp4")

audio = video.audio

audio.write_audiofile("Churchil.wav")

chunk:   0%|                                                                | 3/3182 [00:00<02:26, 21.69it/s, now=None]

MoviePy - Writing audio in Churchil.wav


MoviePy - Done.


## Convert .wav audio to mono

In [5]:
file_name="Churchil.wav"

frame_rate, channels = frame_rate_channel(file_name)
    
if channels > 1:
    stereo_to_mono(file_name)

##  Uses speechsdk to get the transcript

In [9]:
speech_config = speechsdk.SpeechConfig(subscription="", region="")

By default speech sdk returns only 15 seconds of the transcript, this custom function returns the transcript for a long audio

In [10]:


def regcognized(evt):
    print(evt.result.text)
    f= open( "churchill.txt","a")
    f.write(evt.result.text)
    f.close()

def speech_recognize_continuous_from_file():
    """performs continuous speech recognition with input from an audio file"""
    speech_config = speechsdk.SpeechConfig(subscription="", region="")
    audio_config = speechsdk.audio.AudioConfig(filename="Churchil.wav")
 
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, language="en-GB",audio_config=audio_config)
 
    done = False
 
    def stop_cb(evt):
        """callback that stops continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        speech_recognizer.stop_continuous_recognition()
        nonlocal done
        done = True
        
    def split(evt):
        st = re.search(r'\".+?\"',str(evt))
        print(st.group(0).strip('"'))
 

    speech_recognizer.recognized.connect(lambda evt: regcognized((evt)))

    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)
   # speech_recognizer.recognized.connect(split)
 
    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

In [11]:
speech_recognize_continuous_from_file()


When Napoleon led Blue line for a year with the fat bottom boats.
And in Grand Army he was told by someone.
There are bitter weeds in England.
There are certainly a great many more of them since the British Expeditionary Force returned.
So I have myself full confidence that you've all do their duty.
If nothing is neglected, and if the best arrangements are made and they are being made, we should prove ourselves once more able to defend our island home.
Ride out the storm of war.
Man, do I believe the menace of tyranny.
If necessary, via's.
If necessary alone.
Generate Daddy Broadway going to try to do.
That is a result of His Majesty's Government everyman of them.
That is the will of Parliament in the nation, the British Empire and the French Republic linked together in their course and in their need.
Will defend to the death your native soil eating each other like good Cambridge to the utmost of their strings. We shall go onto the end.
Mayfield fighting from.
We will fight on the seas

## Transcript result

In [12]:
f = open("churchill.txt", "r")
print(f.read())

We shall never surrender, and if we could, I do not, for a moment believe.Little Island or large part of it was subjugated and starving.Then our empire beyond the seas armed and guarded by the blue feet.When Napoleon led Blue line for a year with the fat bottom boats.And in Grand Army he was told by someone.There are bitter weeds in England.There are certainly a great many more of them since the British Expeditionary Force returned.So I have myself full confidence that you've all do their duty.If nothing is neglected, and if the best arrangements are made and they are being made, we should prove ourselves once more able to defend our island home.Ride out the storm of war.Man, do I believe the menace of tyranny.If necessary, via's.If necessary alone.Generate Daddy Broadway going to try to do.That is a result of His Majesty's Government everyman of them.That is the will of Parliament in the nation, the British Empire and the French Republic linked together in their course and in their ne

## Translate English transcript to spanish

In [38]:
import requests, uuid, json

# Add your subscription key and endpoint
subscription_key = ""
endpoint = "https://api.cognitive.microsofttranslator.com"

f = open("churchill.txt", "r")
English_speech = f.read()

# Add your location, also known as region. The default is global.
# This is required if using a Cognitive Services resource.
location = "eastus"

path = '/translate'
constructed_url = endpoint + path

params = {
    'api-version': '3.0',
    'from': 'en',
    'to': ['es']
}
constructed_url = endpoint + path

headers = {
    'Ocp-Apim-Subscription-Key': subscription_key,
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

# You can pass more than one object in body.
body = [{
    'text': English_speech
}]

request = requests.post(constructed_url, params=params, headers=headers, json=body)
response = request.json()


In [39]:
trans = json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': '))

In [40]:
resp_dict = json.loads(trans)

spanish_speech = resp_dict[0]["translations"][0]["text"]

Spanish Translation

In [41]:
print(spanish_speech)

Nunca nos rendiremos, y si pudiéramos, no lo creo, por un momento. Little Island o gran parte de ella fue subyugado y hambriento. Entonces nuestro imperio más allá de los mares armado y custodiado por los pies azules. Cuando Napoleón dirigió la línea azul durante un año con los barcos gordos de fondo. Y en el Gran Ejército alguien le dijo. Hay hierbas amargas en Inglaterra.Ciertamente hay muchas más de ellas desde que la Fuerza Expedicionaria Británica regresó. Así que tengo plena confianza en que todos ustedes han cumplir con su deber. Si no se descuida nada, y si se hacen los mejores arreglos y se están haciendo, deberíamos demostrar que una vez más somos capaces de defender nuestra isla de casa. Sal de la tormenta de la guerra. Hombre, creo que la amenaza de la tiranía. Si es necesario, a través de. Si es necesario solo. Generar papá Broadway va a tratar de hacer. Eso es el resultado del Gobierno de Su Majestad cada uno de ellos. Esa es la voluntad del Parlamento en la nación, el Im

## SpeechSynthesizer, spanish text to voice

In [69]:
def speech_synthesis_to_wave_file(spanish_speech):
    """performs speech synthesis to a wave file"""
    # Creates an instance of a speech config with specified subscription key and service region.
    speech_config = speechsdk.SpeechConfig(subscription="", region="")
    
    language = "es-ES";
    speech_config.speech_synthesis_language = language
    
    # Creates a speech synthesizer using file as audio output.
    # Replace with your own audio file name.
    file_name = "outputaudio_spanish.wav"
    file_config = speechsdk.audio.AudioOutputConfig(filename=file_name)
    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=file_config)


    result = speech_synthesizer.speak_text_async(spanish_speech).get()



In [70]:
speech_synthesis_to_wave_file(spanish_speech)

## Play Final result

In [71]:
from pygame import mixer  # Load the popular external library

mixer.init()
mixer.music.load('outputaudio_spanish.wav')
mixer.music.play()

pygame 2.0.1 (SDL 2.0.14, Python 3.7.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
